# Parallel Computing

Before you dive into this, let me just tell you the punchline of this entire page right up front: parallelism is the **last tool** you want to turn to for speed. It is **not** a silver bullet, it will generally take you *significant* time to implement, the speed improvements from parallelism are generally **much** smaller than what you get from other performance improvement methods (see [Understanding Performance](performance_understanding.ipynb) and [Performance Solutions](performance_solutions.ipynb)), and the headaches of parallelizing code are many. 

## What *is* Parallelism

Parallelism is the process of:

1. taking a single problem, 
2. breaking it into lots of smaller problems, 
3. assigning those smaller problems to a number of processing cores that are able to operate independently, and
4. recombining the results. 

As this list shows, parallelism is not easy, and so not only does it take substantial developer time (the time it takes you to implement it), but there are computer-time costs to breaking down problems, distributing them, and recombining them, often limiting the returns you will see to parallelism. 

## Why is Paralleism important?

Given all that, why is parallelism all the range?

The simple answer is that, when it comes to running a serial program (a problem where you run your code in sequence, one step at a time), processors stop getting faster in about the mid-2000s. 

This might surprise you. Most of us have heard that Moore's Law dictates that processors are doubling in performance every 18 months. The reality, however, is more complicated. 

Moore's law used to apply to a number of aspect of processors: the size of transistors, the number of transisters, and the speed that a processor executed serial code. But as shown in the figure below, processor frequency and the speed of serial execution stopped this pattern of doubling in the mid-2000s (serial execution has still been making small gains since then, but even that is iffy -- those improvements are due to little hacks that only work when programs work in just the right way). 

![42-years-processor-trend](images/42-years-processor-trend.png)

Source: [karlrupp.net](https://www.karlrupp.net/2018/02/42-years-of-microprocessor-trend-data/)

And so since chip makers have lost the ability to make their processors faster, they've turned to just giving us more and more processors in the form or more "cores": components of chips that are capable of independent operation. And leveraging the availability of massive numbers of cores (either in single machines, or for huge datasets, across lots of computers) has become the only place left to go for performance!


## Limits of Parallelism

OK, now let's talk about the theoretical limits of parallelism. 

The biggest problem with parallelism is that it's very hard to break some problems into smaller pieces you can work on simultaneously. 
Some thing you do on computers are fundamentally *serial* / *sequential*, and thus cannot be broken up.

For example, suppose you want to simulate how weather evolves over time. The only way to simulate how weather will evolve on day 2 of your simulation is to wait till you've finished simulating day 1 so you can use those results as the starting point for your simulation in day 2. That means there's no way to fully parallelize a weather simulation, since the results at time $t$ will always depend on the results generated at time $t-1$. 

To be clear, this is not to say there are *no* opportunities to speed up weather simulations -- since you general run weather simulations over and over and then look at the *average* prediction, each separate simulation can be parallelized. But it does mean that *even if you had infinite processors*, you couldn't bring the time it takes to simulate the weather down to zero because you'd have to finish simulating day 1 before you can simulate day 2. 

### Amdahl's Law

The formalization of this idea is what's called Amdahl's Law, which gives the *theoretical* limits of parallelization. If $P$ is the proportion of your algorithm that can be parallelized, the fundamental limit to the speed-up you can get from parallelization is given by:

$$\frac{1}{(1-P)+\frac{P}{N}}$$

Expressed graphically, this is:

![amdahlslaw](images/amdahlslaw.png)

Source: [Wikipedia](https://en.wikipedia.org/wiki/Amdahl%27s_law)

As this figure shows, even for a tasks that is *95\% parallelizable*, the biggest possible performance gain you will ever get (with infinite cores), even ignoring any real-world overheads required to execute that parallelization is 20x. 

Now, to be clear, that doesn't mean there aren't situations where the best strategy is parallelism. If you've exhausted all your other opportunities to speed up your code, parallelism may be all that's left. And in data science, it's not uncommon to have code that can be much more than 95\% parallelizable -- for example, if you need to run a simulation 1,000,000 times, and each run is relatively short, you can get close to 100\% parallelizable. But the point is that parallelism is no silver bullet, and it's important to think hard about whether your problem is suited to parallelizing before you invest in trying to parallelize your code!

## Vocabulary 

The terms processor and core are often used interchangable among computer scientists to refer to "units capable of indepedent operation". This can be confusing because most of us think of a "processor" as a single square piece of silicone material that is plugged into our motherboard (e.g. a Intel i9, or an Intel i7):

![intel_chip](images/intel_chip.jpg)

But what most of us think of as processors today often have many cores (the Intel i9 on my laptop has 8 cores, each capable of working relatively independently). Here's a labeled image of the inside of a core i7 processor with four distinct "cores":

![core_i7](images/core_i7.gif)

This gets even more confusing for two reasons:

First, many desktops have two seperate chips (i.e. two Intel i7s, or two Intel i9s), leading some people to call these multi-processor machines. :/

Second, many modern processors have a feature called "hyperthreading", which is where two tasks can be assigned to the same individual core. That core can't do both tasks at the same time, but it can try and switch between them efficiently. For example, if the first task stalls out while it's waiting to get more data from memory (remember how [slow memory is](performance_understanding.ipynb)), the core can switch to working on the second task instead of just sitting around. This *can* offer additional performance, but much less than actually having an actual additional independent core, and actual performance depends on the nature of the job being run (I've heard data science people say if two cores gets you 2x performance, then hyperthreading gets you about 1.25x performance, which seems roughly consistent with my experience. But again this depends *hugely* on the nature of the job you're parallelizing). 

Confusingly, though, the way hyperthreading manifests is by telling your operating system that you have two cores for every physical core on your computer. So for example, while my computer has only 8 physical cores, if I open Python and run:

In [2]:
import os
os.cpu_count()                                                                                                                                 

16

I get 16, because each physical core is presenting itself as two cores to my operating system. 

So, how you you think about this?

- In most contexts (i.e. if you're not shoping for hardware), expect the terms "cores" and "processors" to both be used to refer to independent processing cores. 
- Don't worry about whether they're distributed across two different physical chips or not -- all that matters is the number of cores on your computer. 
- On most computers, expect the operating system to think that you have twice as many cores (sometimes referred to as "logical cores") as you have actual "physical cores", but expect your performance to reflect the number of physical cores you actually have. 

## Types of Parallelism

### Multi-processing

### Multi-threading

DANGER! HERE BE DRAGONS! RACE CONDITIONS!

## How do I paralleize?

joblib

https://sebastianraschka.com/Articles/2014_multiprocessing.html

https://wiki.python.org/moin/ParallelProcessing

dask

Further reading / resources: 

https://www.youtube.com/watch?v=zX4ZNfvw1cw